!tar -czf sourcedir.tar.gz .



!aws s3 cp sourcedir.tar.gz s3://poison-detect-bucket/submission2/scripts/sourcedir.tar.gz

In [17]:
## Looking for tau parameters for backdoor trigger 
import sys, os
sys.path.append(os.path.abspath(".."))
from Detection_tools import utils as UT
import numpy as np 

print(UT)
m = 200                     # size of trigger eval set
p0 = 1 / 10                 # baseline hit rate (10 classes)
f_fp = 0.01                 # 1% false positive rate

tau_stat = UT.statistical_bound_threshold(m, p0, f_fp)
print(f"Statistical bound threshold (τ): {tau_stat:.3f}")

benign_scores = np.random.beta(2, 10, size=50)  # fake benign S scores
tau_emp = UT.empirical_quantile_threshold(benign_scores, 0.95)
print(f"Empirical quantile threshold (τ): {tau_emp:.3f}")


<module 'Detection_tools.utils' from '/home/sagemaker-user/Stinging_Nettle_Prototype/Detection_tools/utils.py'>
Statistical bound threshold (τ): 0.150
Empirical quantile threshold (τ): 0.325


# 1. Label Flipping method : 

## 1.1 Scenario I :
Single Client targeted ( client number 1) + verified clients number 0, 1, 2

### 1.1.1 20 clients:

#### 1.1.1.1 20 clients + Fedopt

In [15]:

from sagemaker.huggingface import HuggingFace

role = "arn:aws:iam::711387130895:role/service-role/SageMaker-ExecutionRole-20250525T120088"

# Define hyperparameters that match your argparse arguments
hyperparameters = {
    # --- Dataset and client setup ---
    "dataset_name": "mnist",
    "num_clients": 20,
    "frac_clients": 1.0,
    "dirichlet_alpha": 0.5,
    "test_size": 0.2,
    "poison_size": 0.35,

    # --- Training parameters ---
    "rounds": 10,
    "local_epochs": 3,
    "batch_size": 32, # from cifar10 64 
    "client_lr": 0.01,
    "client_momentum": 0.9,
    "weight_decay": 1e-4,

    # --- Server optimisation ---
    "server_opt": "fedopt",
    "server_lr": 0.7,

    # --- Fingerprinting ---
    "enable_fingerprinting": "1",          # or False if disabled
    "fingerprint_method": "sparse",         # choices: ["sparse", "dense"]
    "fingerprint_sparsity": 0.01,
    "target_dot_strength": 1.0,

    # --- Detection / Defence params ---
    "honest_fraction": 0.1,
    "detection_margin": 1.5,
    "seed": 42,
    "history_window": 5,
    "method": "label_flip",                 # choices: ["label_flip", "backdoor","fingerprint"]
    "label_flip_alpha": 1.0,
    "backdoor_target_label": 1,
    "backdoor_patch_size": 15,
    "backdoor_intensity": 1.0,
    "tau_backdoor_threshold_statistical": tau_stat,
    "tau_backdoor_threshold_emprical": tau_emp,


    # --- Client selection ---
    # Lists must be passed as space-separated strings for SageMaker
    "targeted_clients": "1",
    "verified_clients": "0 1 2",
}


huggingface_estimator = HuggingFace(
    entry_point="main.py",   # script inside source_dir
    source_dir="s3://poison-detect-bucket/submission2/scripts/sourcedir.tar.gz",  # zipped dir
    role=role,
    instance_type="ml.g4dn.12xlarge",  # use quota for faster training
    instance_count=1,
    transformers_version="4.36.0",
    pytorch_version="2.1.0",
    py_version="py310",
    base_job_name="label-flip-scenario1-fedopt-20clients",
    hyperparameters=hyperparameters,  # pass args here
)

# Launch training
huggingface_estimator.fit(wait=False)


INFO:sagemaker.telemetry.telemetry_logging:SageMaker Python SDK will collect telemetry to help us better understand our user's needs, diagnose issues, and deliver additional features.
To opt out of telemetry, please disable via TelemetryOptOut parameter in SDK defaults config. For more information, refer to https://sagemaker.readthedocs.io/en/stable/overview.html#configuring-and-using-defaults-with-the-sagemaker-python-sdk.
INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: label-flip-scenario1-fedopt-20clients-2025-11-14-17-28-12-438


#### 1.1.1.2 20 clients + FedAVG

In [5]:

from sagemaker.huggingface import HuggingFace

role = "arn:aws:iam::711387130895:role/service-role/SageMaker-ExecutionRole-20250525T120088"

# Define hyperparameters that match your argparse arguments
hyperparameters = {
    # --- Dataset and client setup ---
    "dataset_name": "mnist",
    "num_clients": 20,
    "frac_clients": 1.0,
    "dirichlet_alpha": 0.5,
    "test_size": 0.2,
    "poison_size": 0.35,

    # --- Training parameters ---
    "rounds": 10,
    "local_epochs": 3,
    "batch_size": 32, # from cifar10 64 
    "client_lr": 0.01,
    "client_momentum": 0.9,
    "weight_decay": 1e-4,

    # --- Server optimisation ---
    "server_opt": "fedavg",
    "server_lr": 0.7,

    # --- Fingerprinting ---
    "enable_fingerprinting": "1",          # or False if disabled
    "fingerprint_method": "sparse",         # choices: ["sparse", "dense"]
    "fingerprint_sparsity": 0.01,
    "target_dot_strength": 1.0,

    # --- Detection / Defence params ---
    "honest_fraction": 0.1,
    "detection_margin": 1.5,
    "seed": 42,
    "history_window": 5,
    "method": "label_flip",                 # choices: ["label_flip", "backdoor","fingerprint"]
    "label_flip_alpha": 1.0,
    "backdoor_target_label": 1,
    "backdoor_patch_size": 15,
    "backdoor_intensity": 1.0,
    "tau_backdoor_threshold_statistical": tau_stat,
    "tau_backdoor_threshold_emprical": tau_emp,


    # --- Client selection ---
    # Lists must be passed as space-separated strings for SageMaker
    "targeted_clients": "1",
    "verified_clients": "0 1 2",
}


huggingface_estimator = HuggingFace(
    entry_point="main.py",   # script inside source_dir
    source_dir="s3://poison-detect-bucket/submission2/scripts/sourcedir.tar.gz",  # zipped dir
    role=role,
    instance_type="ml.g4dn.12xlarge",  # use quota for faster training
    instance_count=1,
    transformers_version="4.36.0",
    pytorch_version="2.1.0",
    py_version="py310",
    base_job_name="label-flip-scenario1-fedavg-20clients",
    hyperparameters=hyperparameters,  # pass args here
)

# Launch training
huggingface_estimator.fit(wait=False)


INFO:sagemaker.telemetry.telemetry_logging:SageMaker Python SDK will collect telemetry to help us better understand our user's needs, diagnose issues, and deliver additional features.
To opt out of telemetry, please disable via TelemetryOptOut parameter in SDK defaults config. For more information, refer to https://sagemaker.readthedocs.io/en/stable/overview.html#configuring-and-using-defaults-with-the-sagemaker-python-sdk.
INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: label-flip-scenario1-fedavg-20clients-2025-11-15-10-49-35-481


### 1.1.2 50 clients:

#### 1.1.2.1 50 clients + Fedopt

In [2]:

from sagemaker.huggingface import HuggingFace

role = "arn:aws:iam::711387130895:role/service-role/SageMaker-ExecutionRole-20250525T120088"

# Define hyperparameters that match your argparse arguments
hyperparameters = {
    # --- Dataset and client setup ---
    "dataset_name": "mnist",
    "num_clients": 50,
    "frac_clients": 1.0,
    "dirichlet_alpha": 0.5,
    "test_size": 0.2,
    "poison_size": 0.35,

    # --- Training parameters ---
    "rounds": 12,
    "local_epochs": 3,
    "batch_size": 64, # from cifar10 64 
    "client_lr": 0.006,
    "client_momentum": 0.9,
    "weight_decay": 1e-4,

    # --- Server optimisation ---
    "server_opt": "fedopt",
    "server_lr": 0.35,

    # --- Fingerprinting ---
    "enable_fingerprinting": "1",          # or False if disabled
    "fingerprint_method": "sparse",         # choices: ["sparse", "dense"]
    "fingerprint_sparsity": 0.01,
    "target_dot_strength": 1.0,

    # --- Detection / Defence params ---
    "honest_fraction": 0.1,
    "detection_margin": 1.5,
    "seed": 42,
    "history_window": 5,
    "method": "label_flip",                 # choices: ["label_flip", "backdoor","fingerprint"]
    "label_flip_alpha": 1.0,
    "backdoor_target_label": 1,
    "backdoor_patch_size": 15,
    "backdoor_intensity": 1.0,
    "tau_backdoor_threshold_statistical": tau_stat,
    "tau_backdoor_threshold_emprical": tau_emp,


    # --- Client selection ---
    # Lists must be passed as space-separated strings for SageMaker
    "targeted_clients": "1",
    "verified_clients": "0 1 2",
}


huggingface_estimator = HuggingFace(
    entry_point="main.py",   # script inside source_dir
    source_dir="s3://poison-detect-bucket/submission2/scripts/sourcedir.tar.gz",  # zipped dir
    role=role,
    instance_type="ml.g4dn.12xlarge",  # use quota for faster training
    instance_count=1,
    transformers_version="4.36.0",
    pytorch_version="2.1.0",
    py_version="py310",
    base_job_name="label-flip-scenario1-fedopt-50clients-mnist",
    hyperparameters=hyperparameters,  # pass args here
)

# Launch training
huggingface_estimator.fit(wait=False)


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: label-flip-scenario1-fedopt-50clients-m-2025-11-15-16-50-14-970


#### 1.1.2.1 50 clients + FedAVG

In [3]:

from sagemaker.huggingface import HuggingFace

role = "arn:aws:iam::711387130895:role/service-role/SageMaker-ExecutionRole-20250525T120088"

# Define hyperparameters that match your argparse arguments
hyperparameters = {
    # --- Dataset and client setup ---
    "dataset_name": "mnist",
    "num_clients": 50,
    "frac_clients": 1.0,
    "dirichlet_alpha": 0.5,
    "test_size": 0.2,
    "poison_size": 0.35,

    # --- Training parameters ---
    "rounds": 12,
    "local_epochs": 3,
    "batch_size": 64, # from cifar10 64 
    "client_lr": 0.006,
    "client_momentum": 0.9,
    "weight_decay": 1e-4,

    # --- Server optimisation ---
    "server_opt": "fedavg",
    "server_lr": 0.35,

    # --- Fingerprinting ---
    "enable_fingerprinting": "1",          # or False if disabled
    "fingerprint_method": "sparse",         # choices: ["sparse", "dense"]
    "fingerprint_sparsity": 0.01,
    "target_dot_strength": 1.0,

    # --- Detection / Defence params ---
    "honest_fraction": 0.1,
    "detection_margin": 1.5,
    "seed": 42,
    "history_window": 5,
    "method": "label_flip",                 # choices: ["label_flip", "backdoor","fingerprint"]
    "label_flip_alpha": 1.0,
    "backdoor_target_label": 1,
    "backdoor_patch_size": 15,
    "backdoor_intensity": 1.0,
    "tau_backdoor_threshold_statistical": tau_stat,
    "tau_backdoor_threshold_emprical": tau_emp,


    # --- Client selection ---
    # Lists must be passed as space-separated strings for SageMaker
    "targeted_clients": "1",
    "verified_clients": "0 1 2",
}


huggingface_estimator = HuggingFace(
    entry_point="main.py",   # script inside source_dir
    source_dir="s3://poison-detect-bucket/submission2/scripts/sourcedir.tar.gz",  # zipped dir
    role=role,
    instance_type="ml.g4dn.12xlarge",  # use quota for faster training
    instance_count=1,
    transformers_version="4.36.0",
    pytorch_version="2.1.0",
    py_version="py310",
    base_job_name="label-flip-scenario1-fedavg-50clients-mnist",
    hyperparameters=hyperparameters,  # pass args here
)

# Launch training
huggingface_estimator.fit(wait=False)


INFO:sagemaker.telemetry.telemetry_logging:SageMaker Python SDK will collect telemetry to help us better understand our user's needs, diagnose issues, and deliver additional features.
To opt out of telemetry, please disable via TelemetryOptOut parameter in SDK defaults config. For more information, refer to https://sagemaker.readthedocs.io/en/stable/overview.html#configuring-and-using-defaults-with-the-sagemaker-python-sdk.
INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: label-flip-scenario1-fedavg-50clients-m-2025-11-15-19-45-37-327


### 1.1.3 100 clients:

#### 1.1.3.1 100 clients + FedOPt

In [7]:

from sagemaker.huggingface import HuggingFace

role = "arn:aws:iam::711387130895:role/service-role/SageMaker-ExecutionRole-20250525T120088"

# Define hyperparameters that match your argparse arguments
hyperparameters = {
    # --- Dataset and client setup ---
    "dataset_name": "mnist",
    "num_clients": 100,
    "frac_clients": 1.0,
    "dirichlet_alpha": 0.5,
    "test_size": 0.2,
    "poison_size": 0.35,

    # --- Training parameters ---
    "rounds": 7,
    "local_epochs": 4,
    "batch_size": 8, # from cifar10 64 
    "client_lr": 0.025,
    "client_momentum": 0.9,
    "weight_decay": 1e-4,

    # --- Server optimisation ---
    "server_opt": "fedopt",
    "server_lr": 0.18,

    # --- Fingerprinting ---
    "enable_fingerprinting": "1",          # or False if disabled
    "fingerprint_method": "sparse",         # choices: ["sparse", "dense"]
    "fingerprint_sparsity": 0.01,
    "target_dot_strength": 1.0,

    # --- Detection / Defence params ---
    "honest_fraction": 0.1,
    "detection_margin": 1.5,
    "seed": 42,
    "history_window": 5,
    "method": "label_flip",                 # choices: ["label_flip", "backdoor","fingerprint"]
    "label_flip_alpha": 1.0,
    "backdoor_target_label": 1,
    "backdoor_patch_size": 15,
    "backdoor_intensity": 1.0,
    "tau_backdoor_threshold_statistical": tau_stat,
    "tau_backdoor_threshold_emprical": tau_emp,


    # --- Client selection ---
    # Lists must be passed as space-separated strings for SageMaker
    "targeted_clients": "1",
    "verified_clients": "0 1 2",
}


huggingface_estimator = HuggingFace(
    entry_point="main.py",   # script inside source_dir
    source_dir="s3://poison-detect-bucket/submission2/scripts/sourcedir.tar.gz",  # zipped dir
    role=role,
    instance_type="ml.g4dn.xlarge",  # use quota for faster training
    instance_count=1,
    transformers_version="4.36.0",
    pytorch_version="2.1.0",
    py_version="py310",
    base_job_name="label-flip-scenario1-fedOPT-100clients",
    hyperparameters=hyperparameters,  # pass args here
)

# Launch training
huggingface_estimator.fit(wait=False)


INFO:sagemaker.telemetry.telemetry_logging:SageMaker Python SDK will collect telemetry to help us better understand our user's needs, diagnose issues, and deliver additional features.
To opt out of telemetry, please disable via TelemetryOptOut parameter in SDK defaults config. For more information, refer to https://sagemaker.readthedocs.io/en/stable/overview.html#configuring-and-using-defaults-with-the-sagemaker-python-sdk.
INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: label-flip-scenario1-fedOPT-100clients-2025-11-08-14-27-44-533


### 1.1.3 100 clients:

#### 1.1.3.2 100 clients + FedAVG


In [8]:

from sagemaker.huggingface import HuggingFace

role = "arn:aws:iam::711387130895:role/service-role/SageMaker-ExecutionRole-20250525T120088"

# Define hyperparameters that match your argparse arguments
hyperparameters = {
    # --- Dataset and client setup ---
    "dataset_name": "mnist",
    "num_clients": 100,
    "frac_clients": 1.0,
    "dirichlet_alpha": 0.5,
    "test_size": 0.2,
    "poison_size": 0.35,

    # --- Training parameters ---
    "rounds": 7,
    "local_epochs": 4,
    "batch_size": 8, # from cifar10 64 
    "client_lr": 0.025,
    "client_momentum": 0.9,
    "weight_decay": 1e-4,

    # --- Server optimisation ---
    "server_opt": "fedavg",
    "server_lr": 0.18,

    # --- Fingerprinting ---
    "enable_fingerprinting": "1",          # or False if disabled
    "fingerprint_method": "sparse",         # choices: ["sparse", "dense"]
    "fingerprint_sparsity": 0.01,
    "target_dot_strength": 1.0,

    # --- Detection / Defence params ---
    "honest_fraction": 0.1,
    "detection_margin": 1.5,
    "seed": 42,
    "history_window": 5,
    "method": "label_flip",                 # choices: ["label_flip", "backdoor","fingerprint"]
    "label_flip_alpha": 1.0,
    "backdoor_target_label": 1,
    "backdoor_patch_size": 15,
    "backdoor_intensity": 1.0,
    "tau_backdoor_threshold_statistical": tau_stat,
    "tau_backdoor_threshold_emprical": tau_emp,


    # --- Client selection ---
    # Lists must be passed as space-separated strings for SageMaker
    "targeted_clients": "1",
    "verified_clients": "0 1 2",
}


huggingface_estimator = HuggingFace(
    entry_point="main.py",   # script inside source_dir
    source_dir="s3://poison-detect-bucket/submission2/scripts/sourcedir.tar.gz",  # zipped dir
    role=role,
    instance_type="ml.g5.xlarge",  # use quota for faster training
    instance_count=1,
    transformers_version="4.36.0",
    pytorch_version="2.1.0",
    py_version="py310",
    base_job_name="label-flip-scenario1-fedAVG-100clients",
    hyperparameters=hyperparameters,  # pass args here
)

# Launch training
huggingface_estimator.fit(wait=False)


INFO:sagemaker.telemetry.telemetry_logging:SageMaker Python SDK will collect telemetry to help us better understand our user's needs, diagnose issues, and deliver additional features.
To opt out of telemetry, please disable via TelemetryOptOut parameter in SDK defaults config. For more information, refer to https://sagemaker.readthedocs.io/en/stable/overview.html#configuring-and-using-defaults-with-the-sagemaker-python-sdk.
INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: label-flip-scenario1-fedAVG-100clients-2025-11-08-14-28-27-818


## 1.2 Scenario II :
Subset of  clients targeted:
- in the case of 20 clients ( two clients targeted ( 0, 1) + two verified ( 1, 2))
- in the case of 50 clients (four clients targeted (0, 1, 2, 3) + four verified (2, 3, 4, 5))


### 1.2.1 20 clients :


#### 1.2.1.1 Fedopt

In [18]:

from sagemaker.huggingface import HuggingFace

role = "arn:aws:iam::711387130895:role/service-role/SageMaker-ExecutionRole-20250525T120088"

# Define hyperparameters that match your argparse arguments
hyperparameters = {
    # --- Dataset and client setup ---
    "dataset_name": "mnist",
    "num_clients": 20,
    "frac_clients": 1.0,
    "dirichlet_alpha": 0.5,
    "test_size": 0.2,
    "poison_size": 0.35,

    # --- Training parameters ---
    "rounds": 10,
    "local_epochs": 3,
    "batch_size": 32, # from cifar10 64 
    "client_lr": 0.01,
    "client_momentum": 0.9,
    "weight_decay": 1e-4,

    # --- Server optimisation ---
    "server_opt": "fedopt",
    "server_lr": 0.7,

    # --- Fingerprinting ---
    "enable_fingerprinting": "1",          # or False if disabled
    "fingerprint_method": "sparse",         # choices: ["sparse", "dense"]
    "fingerprint_sparsity": 0.01,
    "target_dot_strength": 1.0,

    # --- Detection / Defence params ---
    "honest_fraction": 0.1,
    "detection_margin": 1.5,
    "seed": 42,
    "history_window": 5,
    "method": "label_flip",                 # choices: ["label_flip", "backdoor","fingerprint"]
    "label_flip_alpha": 1.0,
    "backdoor_target_label": 1,
    "backdoor_patch_size": 15,
    "backdoor_intensity": 1.0,
    "tau_backdoor_threshold_statistical": tau_stat,
    "tau_backdoor_threshold_emprical": tau_emp,


    # --- Client selection ---
    # Lists must be passed as space-separated strings for SageMaker
    "targeted_clients": "0 1",
    "verified_clients": "1 2",
}


huggingface_estimator = HuggingFace(
    entry_point="main.py",   # script inside source_dir
    source_dir="s3://poison-detect-bucket/submission2/scripts/sourcedir.tar.gz",  # zipped dir
    role=role,
    instance_type="ml.g4dn.12xlarge",  # use quota for faster training
    instance_count=1,
    transformers_version="4.36.0",
    pytorch_version="2.1.0",
    py_version="py310",
    base_job_name="label-flip-sc2-fedopt-20clients",
    hyperparameters=hyperparameters,  # pass args here
)

# Launch training
huggingface_estimator.fit(wait=False)


INFO:sagemaker.telemetry.telemetry_logging:SageMaker Python SDK will collect telemetry to help us better understand our user's needs, diagnose issues, and deliver additional features.
To opt out of telemetry, please disable via TelemetryOptOut parameter in SDK defaults config. For more information, refer to https://sagemaker.readthedocs.io/en/stable/overview.html#configuring-and-using-defaults-with-the-sagemaker-python-sdk.
INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: label-flip-sc2-fedopt-20clients-2025-11-17-23-52-11-172


#### 1.2.1.2 Fedavg

In [19]:

from sagemaker.huggingface import HuggingFace

role = "arn:aws:iam::711387130895:role/service-role/SageMaker-ExecutionRole-20250525T120088"

# Define hyperparameters that match your argparse arguments
hyperparameters = {
    # --- Dataset and client setup ---
    "dataset_name": "mnist",
    "num_clients": 20,
    "frac_clients": 1.0,
    "dirichlet_alpha": 0.5,
    "test_size": 0.2,
    "poison_size": 0.35,

    # --- Training parameters ---
    "rounds": 10,
    "local_epochs": 3,
    "batch_size": 32, # from cifar10 64 
    "client_lr": 0.01,
    "client_momentum": 0.9,
    "weight_decay": 1e-4,

    # --- Server optimisation ---
    "server_opt": "fedavg",
    "server_lr": 0.7,

    # --- Fingerprinting ---
    "enable_fingerprinting": "1",          # or False if disabled
    "fingerprint_method": "sparse",         # choices: ["sparse", "dense"]
    "fingerprint_sparsity": 0.01,
    "target_dot_strength": 1.0,

    # --- Detection / Defence params ---
    "honest_fraction": 0.1,
    "detection_margin": 1.5,
    "seed": 42,
    "history_window": 5,
    "method": "label_flip",                 # choices: ["label_flip", "backdoor","fingerprint"]
    "label_flip_alpha": 1.0,
    "backdoor_target_label": 1,
    "backdoor_patch_size": 15,
    "backdoor_intensity": 1.0,
    "tau_backdoor_threshold_statistical": tau_stat,
    "tau_backdoor_threshold_emprical": tau_emp,


    # --- Client selection ---
    # Lists must be passed as space-separated strings for SageMaker
    "targeted_clients": "0 1",
    "verified_clients": "1 2",
}


huggingface_estimator = HuggingFace(
    entry_point="main.py",   # script inside source_dir
    source_dir="s3://poison-detect-bucket/submission2/scripts/sourcedir.tar.gz",  # zipped dir
    role=role,
    instance_type="ml.g4dn.12xlarge",  # use quota for faster training
    instance_count=1,
    transformers_version="4.36.0",
    pytorch_version="2.1.0",
    py_version="py310",
    base_job_name="label-flip-sc2-fedavg-20clients",
    hyperparameters=hyperparameters,  # pass args here
)

# Launch training
huggingface_estimator.fit(wait=False)


INFO:sagemaker.telemetry.telemetry_logging:SageMaker Python SDK will collect telemetry to help us better understand our user's needs, diagnose issues, and deliver additional features.
To opt out of telemetry, please disable via TelemetryOptOut parameter in SDK defaults config. For more information, refer to https://sagemaker.readthedocs.io/en/stable/overview.html#configuring-and-using-defaults-with-the-sagemaker-python-sdk.
INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: label-flip-sc2-fedavg-20clients-2025-11-18-00-44-33-436


### 1.2.2 50 clients:

#### 1.2.2.1 Fedopt:

In [20]:

from sagemaker.huggingface import HuggingFace

role = "arn:aws:iam::711387130895:role/service-role/SageMaker-ExecutionRole-20250525T120088"

# Define hyperparameters that match your argparse arguments
hyperparameters = {
    # --- Dataset and client setup ---
    "dataset_name": "mnist",
    "num_clients": 50,
    "frac_clients": 1.0,
    "dirichlet_alpha": 0.5,
    "test_size": 0.2,
    "poison_size": 0.35,

    # --- Training parameters ---
    "rounds": 12,
    "local_epochs": 3,
    "batch_size": 64, # from cifar10 64 
    "client_lr": 0.006,
    "client_momentum": 0.9,
    "weight_decay": 1e-4,

    # --- Server optimisation ---
    "server_opt": "fedopt",
    "server_lr": 0.35,

    # --- Fingerprinting ---
    "enable_fingerprinting": "1",          # or False if disabled
    "fingerprint_method": "sparse",         # choices: ["sparse", "dense"]
    "fingerprint_sparsity": 0.01,
    "target_dot_strength": 1.0,

    # --- Detection / Defence params ---
    "honest_fraction": 0.1,
    "detection_margin": 1.5,
    "seed": 42,
    "history_window": 5,
    "method": "label_flip",                 # choices: ["label_flip", "backdoor","fingerprint"]
    "label_flip_alpha": 1.0,
    "backdoor_target_label": 1,
    "backdoor_patch_size": 15,
    "backdoor_intensity": 1.0,
    "tau_backdoor_threshold_statistical": tau_stat,
    "tau_backdoor_threshold_emprical": tau_emp,


    # --- Client selection ---
    # Lists must be passed as space-separated strings for SageMaker
    "targeted_clients": "0 1 2 3",
    "verified_clients": "2 3 4 5",
}


huggingface_estimator = HuggingFace(
    entry_point="main.py",   # script inside source_dir
    source_dir="s3://poison-detect-bucket/submission2/scripts/sourcedir.tar.gz",  # zipped dir
    role=role,
    instance_type="ml.g4dn.12xlarge",  # use quota for faster training
    instance_count=1,
    transformers_version="4.36.0",
    pytorch_version="2.1.0",
    py_version="py310",
    base_job_name="label-flip-sc2-fedopt-50clients-mnist",
    hyperparameters=hyperparameters,  # pass args here
)

# Launch training
huggingface_estimator.fit(wait=False)


INFO:sagemaker.telemetry.telemetry_logging:SageMaker Python SDK will collect telemetry to help us better understand our user's needs, diagnose issues, and deliver additional features.
To opt out of telemetry, please disable via TelemetryOptOut parameter in SDK defaults config. For more information, refer to https://sagemaker.readthedocs.io/en/stable/overview.html#configuring-and-using-defaults-with-the-sagemaker-python-sdk.
INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: label-flip-sc2-fedopt-50clients-mnist-2025-11-18-00-44-53-523


#### 1.2.2.2 Fedavg

In [21]:

from sagemaker.huggingface import HuggingFace

role = "arn:aws:iam::711387130895:role/service-role/SageMaker-ExecutionRole-20250525T120088"

# Define hyperparameters that match your argparse arguments
hyperparameters = {
    # --- Dataset and client setup ---
    "dataset_name": "mnist",
    "num_clients": 50,
    "frac_clients": 1.0,
    "dirichlet_alpha": 0.5,
    "test_size": 0.2,
    "poison_size": 0.35,

    # --- Training parameters ---
    "rounds": 12,
    "local_epochs": 3,
    "batch_size": 64, # from cifar10 64 
    "client_lr": 0.006,
    "client_momentum": 0.9,
    "weight_decay": 1e-4,

    # --- Server optimisation ---
    "server_opt": "fedavg",
    "server_lr": 0.35,

    # --- Fingerprinting ---
    "enable_fingerprinting": "1",          # or False if disabled
    "fingerprint_method": "sparse",         # choices: ["sparse", "dense"]
    "fingerprint_sparsity": 0.01,
    "target_dot_strength": 1.0,

    # --- Detection / Defence params ---
    "honest_fraction": 0.1,
    "detection_margin": 1.5,
    "seed": 42,
    "history_window": 5,
    "method": "label_flip",                 # choices: ["label_flip", "backdoor","fingerprint"]
    "label_flip_alpha": 1.0,
    "backdoor_target_label": 1,
    "backdoor_patch_size": 15,
    "backdoor_intensity": 1.0,
    "tau_backdoor_threshold_statistical": tau_stat,
    "tau_backdoor_threshold_emprical": tau_emp,


    # --- Client selection ---
    # Lists must be passed as space-separated strings for SageMaker
    "targeted_clients": "0 1 2 3",
    "verified_clients": "2 3 4 5",
}


huggingface_estimator = HuggingFace(
    entry_point="main.py",   # script inside source_dir
    source_dir="s3://poison-detect-bucket/submission2/scripts/sourcedir.tar.gz",  # zipped dir
    role=role,
    instance_type="ml.g4dn.12xlarge",  # use quota for faster training
    instance_count=1,
    transformers_version="4.36.0",
    pytorch_version="2.1.0",
    py_version="py310",
    base_job_name="label-flip-sc2-fedavg-50clients-mnist",
    hyperparameters=hyperparameters,  # pass args here
)

# Launch training
huggingface_estimator.fit(wait=False)


INFO:sagemaker.telemetry.telemetry_logging:SageMaker Python SDK will collect telemetry to help us better understand our user's needs, diagnose issues, and deliver additional features.
To opt out of telemetry, please disable via TelemetryOptOut parameter in SDK defaults config. For more information, refer to https://sagemaker.readthedocs.io/en/stable/overview.html#configuring-and-using-defaults-with-the-sagemaker-python-sdk.
INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: label-flip-sc2-fedavg-50clients-mnist-2025-11-18-00-45-05-950


# 2.Backdoor:

## 2.1 Scenario1: 

### 2.1.1 20 clients

##### 2.1.1.1 Fedopt:

In [6]:

from sagemaker.huggingface import HuggingFace

role = "arn:aws:iam::711387130895:role/service-role/SageMaker-ExecutionRole-20250525T120088"

# Define hyperparameters that match your argparse arguments
hyperparameters = {
    # --- Dataset and client setup ---
    "dataset_name": "mnist",
    "num_clients": 20,
    "frac_clients": 1.0,
    "dirichlet_alpha": 0.5,
    "test_size": 0.2,
    "poison_size": 0.35,

    # --- Training parameters ---
    "rounds": 10,
    "local_epochs": 3,
    "batch_size": 32, # from cifar10 64 
    "client_lr": 0.02,
    "client_momentum": 0.9,
    "weight_decay": 1e-4,

    # --- Server optimisation ---
    "server_opt": "fedopt",
    "server_lr": 0.8,

    # --- Fingerprinting ---
    "enable_fingerprinting": "1",          # or False if disabled
    "fingerprint_method": "sparse",         # choices: ["sparse", "dense"]
    "fingerprint_sparsity": 0.01,
    "target_dot_strength": 1.0,

    # --- Detection / Defence params ---
    "honest_fraction": 0.1,
    "detection_margin": 1.5,
    "seed": 42,
    "history_window": 5,
    "method": "backdoor",                 # choices: ["label_flip", "backdoor","fingerprint"]
    "label_flip_alpha": 1.0,
    "backdoor_target_label": 1,
    "backdoor_patch_size": 15,
    "backdoor_intensity": 1.0,
    "tau_backdoor_threshold_statistical": tau_stat,
    "tau_backdoor_threshold_emprical": tau_emp,


    # --- Client selection ---
    # Lists must be passed as space-separated strings for SageMaker
    "targeted_clients": "1",
    "verified_clients": "0 1 2",
}


huggingface_estimator = HuggingFace(
    entry_point="main.py",   # script inside source_dir
    source_dir="s3://poison-detect-bucket/submission2/scripts/sourcedir.tar.gz",  # zipped dir
    role=role,
    instance_type="ml.g4dn.12xlarge",  # use quota for faster training
    instance_count=1,
    transformers_version="4.36.0",
    pytorch_version="2.1.0",
    py_version="py310",
    base_job_name="backdoor-scenario1-fedopt-20clients-mnist",
    hyperparameters=hyperparameters,  # pass args here
)

# Launch training
huggingface_estimator.fit(wait=False)


INFO:sagemaker.telemetry.telemetry_logging:SageMaker Python SDK will collect telemetry to help us better understand our user's needs, diagnose issues, and deliver additional features.
To opt out of telemetry, please disable via TelemetryOptOut parameter in SDK defaults config. For more information, refer to https://sagemaker.readthedocs.io/en/stable/overview.html#configuring-and-using-defaults-with-the-sagemaker-python-sdk.
INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: backdoor-scenario1-fedopt-20clients-mni-2025-11-15-11-35-12-661


##### 2.1.1.2 Fedavg

In [7]:

from sagemaker.huggingface import HuggingFace

role = "arn:aws:iam::711387130895:role/service-role/SageMaker-ExecutionRole-20250525T120088"

# Define hyperparameters that match your argparse arguments
hyperparameters = {
    # --- Dataset and client setup ---
    "dataset_name": "mnist",
    "num_clients": 20,
    "frac_clients": 1.0,
    "dirichlet_alpha": 0.5,
    "test_size": 0.2,
    "poison_size": 0.35,

    # --- Training parameters ---
    "rounds": 10,
    "local_epochs": 3,
    "batch_size": 32, # from cifar10 64 
    "client_lr": 0.02,
    "client_momentum": 0.9,
    "weight_decay": 1e-4,

    # --- Server optimisation ---
    "server_opt": "fedavg",
    "server_lr": 0.8,

    # --- Fingerprinting ---
    "enable_fingerprinting": "1",          # or False if disabled
    "fingerprint_method": "sparse",         # choices: ["sparse", "dense"]
    "fingerprint_sparsity": 0.01,
    "target_dot_strength": 1.0,

    # --- Detection / Defence params ---
    "honest_fraction": 0.1,
    "detection_margin": 1.5,
    "seed": 42,
    "history_window": 5,
    "method": "backdoor",                 # choices: ["label_flip", "backdoor","fingerprint"]
    "label_flip_alpha": 1.0,
    "backdoor_target_label": 1,
    "backdoor_patch_size": 15,
    "backdoor_intensity": 1.0,
    "tau_backdoor_threshold_statistical": tau_stat,
    "tau_backdoor_threshold_emprical": tau_emp,


    # --- Client selection ---
    # Lists must be passed as space-separated strings for SageMaker
    "targeted_clients": "1",
    "verified_clients": "0 1 2",
}


huggingface_estimator = HuggingFace(
    entry_point="main.py",   # script inside source_dir
    source_dir="s3://poison-detect-bucket/submission2/scripts/sourcedir.tar.gz",  # zipped dir
    role=role,
    instance_type="ml.g4dn.12xlarge",  # use quota for faster training
    instance_count=1,
    transformers_version="4.36.0",
    pytorch_version="2.1.0",
    py_version="py310",
    base_job_name="backdoor-scenario1-fedavg-20clients-mnist",
    hyperparameters=hyperparameters,  # pass args here
)

# Launch training
huggingface_estimator.fit(wait=False)


INFO:sagemaker.telemetry.telemetry_logging:SageMaker Python SDK will collect telemetry to help us better understand our user's needs, diagnose issues, and deliver additional features.
To opt out of telemetry, please disable via TelemetryOptOut parameter in SDK defaults config. For more information, refer to https://sagemaker.readthedocs.io/en/stable/overview.html#configuring-and-using-defaults-with-the-sagemaker-python-sdk.
INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: backdoor-scenario1-fedavg-20clients-mni-2025-11-15-11-35-36-829


### 2.1.2 50 clients 

#### 2.1.2.1 Fedopt

In [4]:

from sagemaker.huggingface import HuggingFace

role = "arn:aws:iam::711387130895:role/service-role/SageMaker-ExecutionRole-20250525T120088"

# Define hyperparameters that match your argparse arguments
hyperparameters = {
    # --- Dataset and client setup ---
    "dataset_name": "mnist",
    "num_clients": 50,
    "frac_clients": 1.0,
    "dirichlet_alpha": 0.5,
    "test_size": 0.2,
    "poison_size": 0.35,

    # --- Training parameters ---
    "rounds": 12,
    "local_epochs": 3,
    "batch_size": 64, # from cifar10 64 
    "client_lr": 0.006,
    "client_momentum": 0.9,
    "weight_decay": 1e-4,

    # --- Server optimisation ---
    "server_opt": "fedopt",
    "server_lr": 0.35,

    # --- Fingerprinting ---
    "enable_fingerprinting": "1",          # or False if disabled
    "fingerprint_method": "sparse",         # choices: ["sparse", "dense"]
    "fingerprint_sparsity": 0.01,
    "target_dot_strength": 1.0,

    # --- Detection / Defence params ---
    "honest_fraction": 0.1,
    "detection_margin": 1.5,
    "seed": 42,
    "history_window": 5,
    "method": "backdoor",                 # choices: ["label_flip", "backdoor","fingerprint"]
    "label_flip_alpha": 1.0,
    "backdoor_target_label": 1,
    "backdoor_patch_size": 15,
    "backdoor_intensity": 1.0,
    "tau_backdoor_threshold_statistical": tau_stat,
    "tau_backdoor_threshold_emprical": tau_emp,


    # --- Client selection ---
    # Lists must be passed as space-separated strings for SageMaker
    "targeted_clients": "1",
    "verified_clients": "0 1 2",
}


huggingface_estimator = HuggingFace(
    entry_point="main.py",   # script inside source_dir
    source_dir="s3://poison-detect-bucket/submission2/scripts/sourcedir.tar.gz",  # zipped dir
    role=role,
    instance_type="ml.g4dn.12xlarge",  # use quota for faster training
    instance_count=1,
    transformers_version="4.36.0",
    pytorch_version="2.1.0",
    py_version="py310",
    base_job_name="backdoor-scenario1-fedopt-50clients-mnist",
    hyperparameters=hyperparameters,  # pass args here
)

# Launch training
huggingface_estimator.fit(wait=False)


INFO:sagemaker.telemetry.telemetry_logging:SageMaker Python SDK will collect telemetry to help us better understand our user's needs, diagnose issues, and deliver additional features.
To opt out of telemetry, please disable via TelemetryOptOut parameter in SDK defaults config. For more information, refer to https://sagemaker.readthedocs.io/en/stable/overview.html#configuring-and-using-defaults-with-the-sagemaker-python-sdk.
INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: backdoor-scenario1-fedopt-50clients-mni-2025-11-15-19-48-12-473


#### 2.1.2.1 FedAvg

In [5]:

from sagemaker.huggingface import HuggingFace

role = "arn:aws:iam::711387130895:role/service-role/SageMaker-ExecutionRole-20250525T120088"

# Define hyperparameters that match your argparse arguments
hyperparameters = {
    # --- Dataset and client setup ---
    "dataset_name": "mnist",
    "num_clients": 50,
    "frac_clients": 1.0,
    "dirichlet_alpha": 0.5,
    "test_size": 0.2,
    "poison_size": 0.35,

    # --- Training parameters ---
    "rounds": 12,
    "local_epochs": 3,
    "batch_size": 64, # from cifar10 64 
    "client_lr": 0.006,
    "client_momentum": 0.9,
    "weight_decay": 1e-4,

    # --- Server optimisation ---
    "server_opt": "fedavg",
    "server_lr": 0.35,

    # --- Fingerprinting ---
    "enable_fingerprinting": "1",          # or False if disabled
    "fingerprint_method": "sparse",         # choices: ["sparse", "dense"]
    "fingerprint_sparsity": 0.01,
    "target_dot_strength": 1.0,

    # --- Detection / Defence params ---
    "honest_fraction": 0.1,
    "detection_margin": 1.5,
    "seed": 42,
    "history_window": 5,
    "method": "backdoor",                 # choices: ["label_flip", "backdoor","fingerprint"]
    "label_flip_alpha": 1.0,
    "backdoor_target_label": 1,
    "backdoor_patch_size": 15,
    "backdoor_intensity": 1.0,
    "tau_backdoor_threshold_statistical": tau_stat,
    "tau_backdoor_threshold_emprical": tau_emp,


    # --- Client selection ---
    # Lists must be passed as space-separated strings for SageMaker
    "targeted_clients": "1",
    "verified_clients": "0 1 2",
}


huggingface_estimator = HuggingFace(
    entry_point="main.py",   # script inside source_dir
    source_dir="s3://poison-detect-bucket/submission2/scripts/sourcedir.tar.gz",  # zipped dir
    role=role,
    instance_type="ml.g4dn.12xlarge",  # use quota for faster training
    instance_count=1,
    transformers_version="4.36.0",
    pytorch_version="2.1.0",
    py_version="py310",
    base_job_name="backdoor-scenario1-fedavg-50clients-mnist",
    hyperparameters=hyperparameters,  # pass args here
)

# Launch training
huggingface_estimator.fit(wait=False)


INFO:sagemaker.telemetry.telemetry_logging:SageMaker Python SDK will collect telemetry to help us better understand our user's needs, diagnose issues, and deliver additional features.
To opt out of telemetry, please disable via TelemetryOptOut parameter in SDK defaults config. For more information, refer to https://sagemaker.readthedocs.io/en/stable/overview.html#configuring-and-using-defaults-with-the-sagemaker-python-sdk.
INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: backdoor-scenario1-fedavg-50clients-mni-2025-11-15-19-49-28-908


## 2.2 Scenario II:

### 2.2.1 20 clients 

#### 2.2.1.1 Fedopt:

In [22]:

from sagemaker.huggingface import HuggingFace

role = "arn:aws:iam::711387130895:role/service-role/SageMaker-ExecutionRole-20250525T120088"

# Define hyperparameters that match your argparse arguments
hyperparameters = {
    # --- Dataset and client setup ---
    "dataset_name": "mnist",
    "num_clients": 20,
    "frac_clients": 1.0,
    "dirichlet_alpha": 0.5,
    "test_size": 0.2,
    "poison_size": 0.35,

    # --- Training parameters ---
    "rounds": 10,
    "local_epochs": 3,
    "batch_size": 32, # from cifar10 64 
    "client_lr": 0.01,
    "client_momentum": 0.9,
    "weight_decay": 1e-4,

    # --- Server optimisation ---
    "server_opt": "fedopt",
    "server_lr": 0.7,

    # --- Fingerprinting ---
    "enable_fingerprinting": "1",          # or False if disabled
    "fingerprint_method": "sparse",         # choices: ["sparse", "dense"]
    "fingerprint_sparsity": 0.01,
    "target_dot_strength": 1.0,

    # --- Detection / Defence params ---
    "honest_fraction": 0.1,
    "detection_margin": 1.5,
    "seed": 42,
    "history_window": 5,
    "method": "backdoor",                 # choices: ["label_flip", "backdoor","fingerprint"]
    "label_flip_alpha": 1.0,
    "backdoor_target_label": 1,
    "backdoor_patch_size": 15,
    "backdoor_intensity": 1.0,
    "tau_backdoor_threshold_statistical": tau_stat,
    "tau_backdoor_threshold_emprical": tau_emp,


    # --- Client selection ---
    # Lists must be passed as space-separated strings for SageMaker
    "targeted_clients": "0 1",
    "verified_clients": "1 2",
}


huggingface_estimator = HuggingFace(
    entry_point="main.py",   # script inside source_dir
    source_dir="s3://poison-detect-bucket/submission2/scripts/sourcedir.tar.gz",  # zipped dir
    role=role,
    instance_type="ml.g4dn.12xlarge",  # use quota for faster training
    instance_count=1,
    transformers_version="4.36.0",
    pytorch_version="2.1.0",
    py_version="py310",
    base_job_name="backdoor-sc2-fedopt-20clients",
    hyperparameters=hyperparameters,  # pass args here
)

# Launch training
huggingface_estimator.fit(wait=False)


INFO:sagemaker.telemetry.telemetry_logging:SageMaker Python SDK will collect telemetry to help us better understand our user's needs, diagnose issues, and deliver additional features.
To opt out of telemetry, please disable via TelemetryOptOut parameter in SDK defaults config. For more information, refer to https://sagemaker.readthedocs.io/en/stable/overview.html#configuring-and-using-defaults-with-the-sagemaker-python-sdk.
INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: backdoor-sc2-fedopt-20clients-2025-11-18-00-46-38-924


#### 2.2.1.2 Fedavg:

In [23]:

from sagemaker.huggingface import HuggingFace

role = "arn:aws:iam::711387130895:role/service-role/SageMaker-ExecutionRole-20250525T120088"

# Define hyperparameters that match your argparse arguments
hyperparameters = {
    # --- Dataset and client setup ---
    "dataset_name": "mnist",
    "num_clients": 20,
    "frac_clients": 1.0,
    "dirichlet_alpha": 0.5,
    "test_size": 0.2,
    "poison_size": 0.35,

    # --- Training parameters ---
    "rounds": 10,
    "local_epochs": 3,
    "batch_size": 32, # from cifar10 64 
    "client_lr": 0.01,
    "client_momentum": 0.9,
    "weight_decay": 1e-4,

    # --- Server optimisation ---
    "server_opt": "fedavg",
    "server_lr": 0.7,

    # --- Fingerprinting ---
    "enable_fingerprinting": "1",          # or False if disabled
    "fingerprint_method": "sparse",         # choices: ["sparse", "dense"]
    "fingerprint_sparsity": 0.01,
    "target_dot_strength": 1.0,

    # --- Detection / Defence params ---
    "honest_fraction": 0.1,
    "detection_margin": 1.5,
    "seed": 42,
    "history_window": 5,
    "method": "backdoor",                 # choices: ["label_flip", "backdoor","fingerprint"]
    "label_flip_alpha": 1.0,
    "backdoor_target_label": 1,
    "backdoor_patch_size": 15,
    "backdoor_intensity": 1.0,
    "tau_backdoor_threshold_statistical": tau_stat,
    "tau_backdoor_threshold_emprical": tau_emp,


    # --- Client selection ---
    # Lists must be passed as space-separated strings for SageMaker
    "targeted_clients": "0 1",
    "verified_clients": "1 2",
}


huggingface_estimator = HuggingFace(
    entry_point="main.py",   # script inside source_dir
    source_dir="s3://poison-detect-bucket/submission2/scripts/sourcedir.tar.gz",  # zipped dir
    role=role,
    instance_type="ml.g4dn.12xlarge",  # use quota for faster training
    instance_count=1,
    transformers_version="4.36.0",
    pytorch_version="2.1.0",
    py_version="py310",
    base_job_name="backdoor-sc2-fedavg-20clients",
    hyperparameters=hyperparameters,  # pass args here
)

# Launch training
huggingface_estimator.fit(wait=False)


INFO:sagemaker.telemetry.telemetry_logging:SageMaker Python SDK will collect telemetry to help us better understand our user's needs, diagnose issues, and deliver additional features.
To opt out of telemetry, please disable via TelemetryOptOut parameter in SDK defaults config. For more information, refer to https://sagemaker.readthedocs.io/en/stable/overview.html#configuring-and-using-defaults-with-the-sagemaker-python-sdk.
INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: backdoor-sc2-fedavg-20clients-2025-11-18-00-46-41-921


### 2.2.2 50 clients

#### 2.2.2.1 Fedopt

In [24]:

from sagemaker.huggingface import HuggingFace

role = "arn:aws:iam::711387130895:role/service-role/SageMaker-ExecutionRole-20250525T120088"

# Define hyperparameters that match your argparse arguments
hyperparameters = {
    # --- Dataset and client setup ---
    "dataset_name": "mnist",
    "num_clients": 50,
    "frac_clients": 1.0,
    "dirichlet_alpha": 0.5,
    "test_size": 0.2,
    "poison_size": 0.35,

    # --- Training parameters ---
    "rounds": 12,
    "local_epochs": 3,
    "batch_size": 64, # from cifar10 64 
    "client_lr": 0.006,
    "client_momentum": 0.9,
    "weight_decay": 1e-4,

    # --- Server optimisation ---
    "server_opt": "fedopt",
    "server_lr": 0.35,

    # --- Fingerprinting ---
    "enable_fingerprinting": "1",          # or False if disabled
    "fingerprint_method": "sparse",         # choices: ["sparse", "dense"]
    "fingerprint_sparsity": 0.01,
    "target_dot_strength": 1.0,

    # --- Detection / Defence params ---
    "honest_fraction": 0.1,
    "detection_margin": 1.5,
    "seed": 42,
    "history_window": 5,
    "method": "backdoor",                 # choices: ["label_flip", "backdoor","fingerprint"]
    "label_flip_alpha": 1.0,
    "backdoor_target_label": 1,
    "backdoor_patch_size": 15,
    "backdoor_intensity": 1.0,
    "tau_backdoor_threshold_statistical": tau_stat,
    "tau_backdoor_threshold_emprical": tau_emp,


    # --- Client selection ---
    # Lists must be passed as space-separated strings for SageMaker
    "targeted_clients": "0 1 2 3",
    "verified_clients": "2 3 4 5",
}


huggingface_estimator = HuggingFace(
    entry_point="main.py",   # script inside source_dir
    source_dir="s3://poison-detect-bucket/submission2/scripts/sourcedir.tar.gz",  # zipped dir
    role=role,
    instance_type="ml.g4dn.12xlarge",  # use quota for faster training
    instance_count=1,
    transformers_version="4.36.0",
    pytorch_version="2.1.0",
    py_version="py310",
    base_job_name="backdoor-sc2-fedopt-50clients-mnist",
    hyperparameters=hyperparameters,  # pass args here
)

# Launch training
huggingface_estimator.fit(wait=False)


INFO:sagemaker.telemetry.telemetry_logging:SageMaker Python SDK will collect telemetry to help us better understand our user's needs, diagnose issues, and deliver additional features.
To opt out of telemetry, please disable via TelemetryOptOut parameter in SDK defaults config. For more information, refer to https://sagemaker.readthedocs.io/en/stable/overview.html#configuring-and-using-defaults-with-the-sagemaker-python-sdk.
INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: backdoor-sc2-fedopt-50clients-mnist-2025-11-18-00-46-45-804


#### 2.2.2.2 Fedavg

In [25]:

from sagemaker.huggingface import HuggingFace

role = "arn:aws:iam::711387130895:role/service-role/SageMaker-ExecutionRole-20250525T120088"

# Define hyperparameters that match your argparse arguments
hyperparameters = {
    # --- Dataset and client setup ---
    "dataset_name": "mnist",
    "num_clients": 50,
    "frac_clients": 1.0,
    "dirichlet_alpha": 0.5,
    "test_size": 0.2,
    "poison_size": 0.35,

    # --- Training parameters ---
    "rounds": 12,
    "local_epochs": 3,
    "batch_size": 64, # from cifar10 64 
    "client_lr": 0.006,
    "client_momentum": 0.9,
    "weight_decay": 1e-4,

    # --- Server optimisation ---
    "server_opt": "fedavg",
    "server_lr": 0.35,

    # --- Fingerprinting ---
    "enable_fingerprinting": "1",          # or False if disabled
    "fingerprint_method": "sparse",         # choices: ["sparse", "dense"]
    "fingerprint_sparsity": 0.01,
    "target_dot_strength": 1.0,

    # --- Detection / Defence params ---
    "honest_fraction": 0.1,
    "detection_margin": 1.5,
    "seed": 42,
    "history_window": 5,
    "method": "backdoor",                 # choices: ["label_flip", "backdoor","fingerprint"]
    "label_flip_alpha": 1.0,
    "backdoor_target_label": 1,
    "backdoor_patch_size": 15,
    "backdoor_intensity": 1.0,
    "tau_backdoor_threshold_statistical": tau_stat,
    "tau_backdoor_threshold_emprical": tau_emp,


    # --- Client selection ---
    # Lists must be passed as space-separated strings for SageMaker
    "targeted_clients": "0 1 2 3",
    "verified_clients": "2 3 4 5",
}


huggingface_estimator = HuggingFace(
    entry_point="main.py",   # script inside source_dir
    source_dir="s3://poison-detect-bucket/submission2/scripts/sourcedir.tar.gz",  # zipped dir
    role=role,
    instance_type="ml.g4dn.12xlarge",  # use quota for faster training
    instance_count=1,
    transformers_version="4.36.0",
    pytorch_version="2.1.0",
    py_version="py310",
    base_job_name="backdoor-sc2-fedavg-50clients-mnist",
    hyperparameters=hyperparameters,  # pass args here
)

# Launch training
huggingface_estimator.fit(wait=False)


INFO:sagemaker.telemetry.telemetry_logging:SageMaker Python SDK will collect telemetry to help us better understand our user's needs, diagnose issues, and deliver additional features.
To opt out of telemetry, please disable via TelemetryOptOut parameter in SDK defaults config. For more information, refer to https://sagemaker.readthedocs.io/en/stable/overview.html#configuring-and-using-defaults-with-the-sagemaker-python-sdk.
INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: backdoor-sc2-fedavg-50clients-mnist-2025-11-18-00-46-48-307


# 3 Fingerprint

## 3.1 Scenario1

### 3.1.1 20 clients

#### 3.1.1.1 Fedopt

In [8]:

from sagemaker.huggingface import HuggingFace

role = "arn:aws:iam::711387130895:role/service-role/SageMaker-ExecutionRole-20250525T120088"

# Define hyperparameters that match your argparse arguments
hyperparameters = {
    # --- Dataset and client setup ---
    "dataset_name": "mnist",
    "num_clients": 20,
    "frac_clients": 1.0,
    "dirichlet_alpha": 0.5,
    "test_size": 0.2,
    "poison_size": 0.35,

    # --- Training parameters ---
    "rounds": 10,
    "local_epochs": 3,
    "batch_size": 32, # from cifar10 64 
    "client_lr": 0.02,
    "client_momentum": 0.9,
    "weight_decay": 1e-4,

    # --- Server optimisation ---
    "server_opt": "fedopt",
    "server_lr": 0.8,

    # --- Fingerprinting ---
    "enable_fingerprinting": "1",          # or False if disabled
    "fingerprint_method": "sparse",         # choices: ["sparse", "dense"]
    "fingerprint_sparsity": 0.01,
    "target_dot_strength": 1.0,

    # --- Detection / Defence params ---
    "honest_fraction": 0.1,
    "detection_margin": 1.5,
    "seed": 42,
    "history_window": 5,
    "method": "fingerprint",                 # choices: ["label_flip", "backdoor","fingerprint"]
    "label_flip_alpha": 1.0,
    "backdoor_target_label": 1,
    "backdoor_patch_size": 15,
    "backdoor_intensity": 1.0,
    "tau_backdoor_threshold_statistical": tau_stat,
    "tau_backdoor_threshold_emprical": tau_emp,


    # --- Client selection ---
    # Lists must be passed as space-separated strings for SageMaker
    "targeted_clients": "1",
    "verified_clients": "0 1 2",
}


huggingface_estimator = HuggingFace(
    entry_point="main.py",   # script inside source_dir
    source_dir="s3://poison-detect-bucket/submission2/scripts/sourcedir.tar.gz",  # zipped dir
    role=role,
    instance_type="ml.g4dn.12xlarge",  # use quota for faster training
    instance_count=1,
    transformers_version="4.36.0",
    pytorch_version="2.1.0",
    py_version="py310",
    base_job_name="fingerprint-scenario1-fedopt-20clients-mnist",
    hyperparameters=hyperparameters,  # pass args here
)

# Launch training
huggingface_estimator.fit(wait=False)


INFO:sagemaker.telemetry.telemetry_logging:SageMaker Python SDK will collect telemetry to help us better understand our user's needs, diagnose issues, and deliver additional features.
To opt out of telemetry, please disable via TelemetryOptOut parameter in SDK defaults config. For more information, refer to https://sagemaker.readthedocs.io/en/stable/overview.html#configuring-and-using-defaults-with-the-sagemaker-python-sdk.
INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: fingerprint-scenario1-fedopt-20clients--2025-11-15-11-36-21-073


#### 3.1.1.2 Fedavg

In [9]:

from sagemaker.huggingface import HuggingFace

role = "arn:aws:iam::711387130895:role/service-role/SageMaker-ExecutionRole-20250525T120088"

# Define hyperparameters that match your argparse arguments
hyperparameters = {
    # --- Dataset and client setup ---
    "dataset_name": "mnist",
    "num_clients": 20,
    "frac_clients": 1.0,
    "dirichlet_alpha": 0.5,
    "test_size": 0.2,
    "poison_size": 0.35,

    # --- Training parameters ---
    "rounds": 10,
    "local_epochs": 3,
    "batch_size": 32, # from cifar10 64 
    "client_lr": 0.02,
    "client_momentum": 0.9,
    "weight_decay": 1e-4,

    # --- Server optimisation ---
    "server_opt": "fedavg",
    "server_lr": 0.8,

    # --- Fingerprinting ---
    "enable_fingerprinting": "1",          # or False if disabled
    "fingerprint_method": "sparse",         # choices: ["sparse", "dense"]
    "fingerprint_sparsity": 0.01,
    "target_dot_strength": 1.0,

    # --- Detection / Defence params ---
    "honest_fraction": 0.1,
    "detection_margin": 1.5,
    "seed": 42,
    "history_window": 5,
    "method": "fingerprint",                 # choices: ["label_flip", "backdoor","fingerprint"]
    "label_flip_alpha": 1.0,
    "backdoor_target_label": 1,
    "backdoor_patch_size": 15,
    "backdoor_intensity": 1.0,
    "tau_backdoor_threshold_statistical": tau_stat,
    "tau_backdoor_threshold_emprical": tau_emp,


    # --- Client selection ---
    # Lists must be passed as space-separated strings for SageMaker
    "targeted_clients": "1",
    "verified_clients": "0 1 2",
}


huggingface_estimator = HuggingFace(
    entry_point="main.py",   # script inside source_dir
    source_dir="s3://poison-detect-bucket/submission2/scripts/sourcedir.tar.gz",  # zipped dir
    role=role,
    instance_type="ml.g4dn.12xlarge",  # use quota for faster training
    instance_count=1,
    transformers_version="4.36.0",
    pytorch_version="2.1.0",
    py_version="py310",
    base_job_name="fingerprint-scenario1-fedavg-20clients-mnist",
    hyperparameters=hyperparameters,  # pass args here
)

# Launch training
huggingface_estimator.fit(wait=False)


INFO:sagemaker.telemetry.telemetry_logging:SageMaker Python SDK will collect telemetry to help us better understand our user's needs, diagnose issues, and deliver additional features.
To opt out of telemetry, please disable via TelemetryOptOut parameter in SDK defaults config. For more information, refer to https://sagemaker.readthedocs.io/en/stable/overview.html#configuring-and-using-defaults-with-the-sagemaker-python-sdk.
INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: fingerprint-scenario1-fedavg-20clients--2025-11-15-11-36-55-809


### 3.1.2 50 clients

#### 3.1.2.1 Fedopt

In [6]:

from sagemaker.huggingface import HuggingFace

role = "arn:aws:iam::711387130895:role/service-role/SageMaker-ExecutionRole-20250525T120088"

# Define hyperparameters that match your argparse arguments
hyperparameters = {
    # --- Dataset and client setup ---
    "dataset_name": "mnist",
    "num_clients": 50,
    "frac_clients": 1.0,
    "dirichlet_alpha": 0.5,
    "test_size": 0.2,
    "poison_size": 0.35,

    # --- Training parameters ---
    "rounds": 12,
    "local_epochs": 3,
    "batch_size": 64, # from cifar10 64 
    "client_lr": 0.006,
    "client_momentum": 0.9,
    "weight_decay": 1e-4,

    # --- Server optimisation ---
    "server_opt": "fedopt",
    "server_lr": 0.35,

    # --- Fingerprinting ---
    "enable_fingerprinting": "1",          # or False if disabled
    "fingerprint_method": "sparse",         # choices: ["sparse", "dense"]
    "fingerprint_sparsity": 0.01,
    "target_dot_strength": 1.0,

    # --- Detection / Defence params ---
    "honest_fraction": 0.1,
    "detection_margin": 1.5,
    "seed": 42,
    "history_window": 5,
    "method": "fingerprint",                 # choices: ["label_flip", "backdoor","fingerprint"]
    "label_flip_alpha": 1.0,
    "backdoor_target_label": 1,
    "backdoor_patch_size": 15,
    "backdoor_intensity": 1.0,
    "tau_backdoor_threshold_statistical": tau_stat,
    "tau_backdoor_threshold_emprical": tau_emp,


    # --- Client selection ---
    # Lists must be passed as space-separated strings for SageMaker
    "targeted_clients": "1",
    "verified_clients": "0 1 2",
}


huggingface_estimator = HuggingFace(
    entry_point="main.py",   # script inside source_dir
    source_dir="s3://poison-detect-bucket/submission2/scripts/sourcedir.tar.gz",  # zipped dir
    role=role,
    instance_type="ml.g4dn.12xlarge",  # use quota for faster training
    instance_count=1,
    transformers_version="4.36.0",
    pytorch_version="2.1.0",
    py_version="py310",
    base_job_name="fingerprint-scenario1-fedopt-50clients-mnist",
    hyperparameters=hyperparameters,  # pass args here
)

# Launch training
huggingface_estimator.fit(wait=False)


INFO:sagemaker.telemetry.telemetry_logging:SageMaker Python SDK will collect telemetry to help us better understand our user's needs, diagnose issues, and deliver additional features.
To opt out of telemetry, please disable via TelemetryOptOut parameter in SDK defaults config. For more information, refer to https://sagemaker.readthedocs.io/en/stable/overview.html#configuring-and-using-defaults-with-the-sagemaker-python-sdk.
INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: fingerprint-scenario1-fedopt-50clients--2025-11-15-19-52-42-385


#### 3.1.2.1 Fedavg

In [7]:

from sagemaker.huggingface import HuggingFace

role = "arn:aws:iam::711387130895:role/service-role/SageMaker-ExecutionRole-20250525T120088"

# Define hyperparameters that match your argparse arguments
hyperparameters = {
    # --- Dataset and client setup ---
    "dataset_name": "mnist",
    "num_clients": 50,
    "frac_clients": 1.0,
    "dirichlet_alpha": 0.5,
    "test_size": 0.2,
    "poison_size": 0.35,

    # --- Training parameters ---
    "rounds": 12,
    "local_epochs": 3,
    "batch_size": 64, # from cifar10 64 
    "client_lr": 0.006,
    "client_momentum": 0.9,
    "weight_decay": 1e-4,

    # --- Server optimisation ---
    "server_opt": "fedavg",
    "server_lr": 0.35,

    # --- Fingerprinting ---
    "enable_fingerprinting": "1",          # or False if disabled
    "fingerprint_method": "sparse",         # choices: ["sparse", "dense"]
    "fingerprint_sparsity": 0.01,
    "target_dot_strength": 1.0,

    # --- Detection / Defence params ---
    "honest_fraction": 0.1,
    "detection_margin": 1.5,
    "seed": 42,
    "history_window": 5,
    "method": "fingerprint",                 # choices: ["label_flip", "backdoor","fingerprint"]
    "label_flip_alpha": 1.0,
    "backdoor_target_label": 1,
    "backdoor_patch_size": 15,
    "backdoor_intensity": 1.0,
    "tau_backdoor_threshold_statistical": tau_stat,
    "tau_backdoor_threshold_emprical": tau_emp,


    # --- Client selection ---
    # Lists must be passed as space-separated strings for SageMaker
    "targeted_clients": "1",
    "verified_clients": "0 1 2",
}


huggingface_estimator = HuggingFace(
    entry_point="main.py",   # script inside source_dir
    source_dir="s3://poison-detect-bucket/submission2/scripts/sourcedir.tar.gz",  # zipped dir
    role=role,
    instance_type="ml.g4dn.12xlarge",  # use quota for faster training
    instance_count=1,
    transformers_version="4.36.0",
    pytorch_version="2.1.0",
    py_version="py310",
    base_job_name="fingerprint-scenario1-fedavg-50clients-mnist",
    hyperparameters=hyperparameters,  # pass args here
)

# Launch training
huggingface_estimator.fit(wait=False)


INFO:sagemaker.telemetry.telemetry_logging:SageMaker Python SDK will collect telemetry to help us better understand our user's needs, diagnose issues, and deliver additional features.
To opt out of telemetry, please disable via TelemetryOptOut parameter in SDK defaults config. For more information, refer to https://sagemaker.readthedocs.io/en/stable/overview.html#configuring-and-using-defaults-with-the-sagemaker-python-sdk.
INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: fingerprint-scenario1-fedavg-50clients--2025-11-15-19-53-09-617


## 3.2 Scenario2

### 3.2.1 20 clients:

#### 3.2.1.1 Fedopt:

In [26]:

from sagemaker.huggingface import HuggingFace

role = "arn:aws:iam::711387130895:role/service-role/SageMaker-ExecutionRole-20250525T120088"

# Define hyperparameters that match your argparse arguments
hyperparameters = {
    # --- Dataset and client setup ---
    "dataset_name": "mnist",
    "num_clients": 20,
    "frac_clients": 1.0,
    "dirichlet_alpha": 0.5,
    "test_size": 0.2,
    "poison_size": 0.35,

    # --- Training parameters ---
    "rounds": 10,
    "local_epochs": 3,
    "batch_size": 32, # from cifar10 64 
    "client_lr": 0.01,
    "client_momentum": 0.9,
    "weight_decay": 1e-4,

    # --- Server optimisation ---
    "server_opt": "fedopt",
    "server_lr": 0.7,

    # --- Fingerprinting ---
    "enable_fingerprinting": "1",          # or False if disabled
    "fingerprint_method": "sparse",         # choices: ["sparse", "dense"]
    "fingerprint_sparsity": 0.01,
    "target_dot_strength": 1.0,

    # --- Detection / Defence params ---
    "honest_fraction": 0.1,
    "detection_margin": 1.5,
    "seed": 42,
    "history_window": 5,
    "method": "fingerprint",                 # choices: ["label_flip", "backdoor","fingerprint"]
    "label_flip_alpha": 1.0,
    "backdoor_target_label": 1,
    "backdoor_patch_size": 15,
    "backdoor_intensity": 1.0,
    "tau_backdoor_threshold_statistical": tau_stat,
    "tau_backdoor_threshold_emprical": tau_emp,


    # --- Client selection ---
    # Lists must be passed as space-separated strings for SageMaker
    "targeted_clients": "0 1",
    "verified_clients": "1 2",
}


huggingface_estimator = HuggingFace(
    entry_point="main.py",   # script inside source_dir
    source_dir="s3://poison-detect-bucket/submission2/scripts/sourcedir.tar.gz",  # zipped dir
    role=role,
    instance_type="ml.g4dn.12xlarge",  # use quota for faster training
    instance_count=1,
    transformers_version="4.36.0",
    pytorch_version="2.1.0",
    py_version="py310",
    base_job_name="fingerprint-sc2-fedopt-20clients",
    hyperparameters=hyperparameters,  # pass args here
)

# Launch training
huggingface_estimator.fit(wait=False)


INFO:sagemaker.telemetry.telemetry_logging:SageMaker Python SDK will collect telemetry to help us better understand our user's needs, diagnose issues, and deliver additional features.
To opt out of telemetry, please disable via TelemetryOptOut parameter in SDK defaults config. For more information, refer to https://sagemaker.readthedocs.io/en/stable/overview.html#configuring-and-using-defaults-with-the-sagemaker-python-sdk.
INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: fingerprint-sc2-fedopt-20clients-2025-11-18-00-46-54-918


#### 3.2.1.2 Fedavg:

In [27]:

from sagemaker.huggingface import HuggingFace

role = "arn:aws:iam::711387130895:role/service-role/SageMaker-ExecutionRole-20250525T120088"

# Define hyperparameters that match your argparse arguments
hyperparameters = {
    # --- Dataset and client setup ---
    "dataset_name": "mnist",
    "num_clients": 20,
    "frac_clients": 1.0,
    "dirichlet_alpha": 0.5,
    "test_size": 0.2,
    "poison_size": 0.35,

    # --- Training parameters ---
    "rounds": 10,
    "local_epochs": 3,
    "batch_size": 32, # from cifar10 64 
    "client_lr": 0.01,
    "client_momentum": 0.9,
    "weight_decay": 1e-4,

    # --- Server optimisation ---
    "server_opt": "fedavg",
    "server_lr": 0.7,

    # --- Fingerprinting ---
    "enable_fingerprinting": "1",          # or False if disabled
    "fingerprint_method": "sparse",         # choices: ["sparse", "dense"]
    "fingerprint_sparsity": 0.01,
    "target_dot_strength": 1.0,

    # --- Detection / Defence params ---
    "honest_fraction": 0.1,
    "detection_margin": 1.5,
    "seed": 42,
    "history_window": 5,
    "method": "fingerprint",                 # choices: ["label_flip", "backdoor","fingerprint"]
    "label_flip_alpha": 1.0,
    "backdoor_target_label": 1,
    "backdoor_patch_size": 15,
    "backdoor_intensity": 1.0,
    "tau_backdoor_threshold_statistical": tau_stat,
    "tau_backdoor_threshold_emprical": tau_emp,


    # --- Client selection ---
    # Lists must be passed as space-separated strings for SageMaker
    "targeted_clients": "0 1",
    "verified_clients": "1 2",
}


huggingface_estimator = HuggingFace(
    entry_point="main.py",   # script inside source_dir
    source_dir="s3://poison-detect-bucket/submission2/scripts/sourcedir.tar.gz",  # zipped dir
    role=role,
    instance_type="ml.g4dn.12xlarge",  # use quota for faster training
    instance_count=1,
    transformers_version="4.36.0",
    pytorch_version="2.1.0",
    py_version="py310",
    base_job_name="fingerprint-sc2-fedavg-20clients",
    hyperparameters=hyperparameters,  # pass args here
)

# Launch training
huggingface_estimator.fit(wait=False)


INFO:sagemaker.telemetry.telemetry_logging:SageMaker Python SDK will collect telemetry to help us better understand our user's needs, diagnose issues, and deliver additional features.
To opt out of telemetry, please disable via TelemetryOptOut parameter in SDK defaults config. For more information, refer to https://sagemaker.readthedocs.io/en/stable/overview.html#configuring-and-using-defaults-with-the-sagemaker-python-sdk.
INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: fingerprint-sc2-fedavg-20clients-2025-11-18-00-46-59-541


### 3.2.2 50 clients


#### 3.2.2.1 Fedopt

In [28]:

from sagemaker.huggingface import HuggingFace

role = "arn:aws:iam::711387130895:role/service-role/SageMaker-ExecutionRole-20250525T120088"

# Define hyperparameters that match your argparse arguments
hyperparameters = {
    # --- Dataset and client setup ---
    "dataset_name": "mnist",
    "num_clients": 50,
    "frac_clients": 1.0,
    "dirichlet_alpha": 0.5,
    "test_size": 0.2,
    "poison_size": 0.35,

    # --- Training parameters ---
    "rounds": 12,
    "local_epochs": 3,
    "batch_size": 64, # from cifar10 64 
    "client_lr": 0.006,
    "client_momentum": 0.9,
    "weight_decay": 1e-4,

    # --- Server optimisation ---
    "server_opt": "fedopt",
    "server_lr": 0.35,

    # --- Fingerprinting ---
    "enable_fingerprinting": "1",          # or False if disabled
    "fingerprint_method": "sparse",         # choices: ["sparse", "dense"]
    "fingerprint_sparsity": 0.01,
    "target_dot_strength": 1.0,

    # --- Detection / Defence params ---
    "honest_fraction": 0.1,
    "detection_margin": 1.5,
    "seed": 42,
    "history_window": 5,
    "method": "fingerprint",                 # choices: ["label_flip", "backdoor","fingerprint"]
    "label_flip_alpha": 1.0,
    "backdoor_target_label": 1,
    "backdoor_patch_size": 15,
    "backdoor_intensity": 1.0,
    "tau_backdoor_threshold_statistical": tau_stat,
    "tau_backdoor_threshold_emprical": tau_emp,


    # --- Client selection ---
    # Lists must be passed as space-separated strings for SageMaker
    "targeted_clients": "0 1 2 3",
    "verified_clients": "2 3 4 5",
}


huggingface_estimator = HuggingFace(
    entry_point="main.py",   # script inside source_dir
    source_dir="s3://poison-detect-bucket/submission2/scripts/sourcedir.tar.gz",  # zipped dir
    role=role,
    instance_type="ml.g4dn.12xlarge",  # use quota for faster training
    instance_count=1,
    transformers_version="4.36.0",
    pytorch_version="2.1.0",
    py_version="py310",
    base_job_name="fingerprint-sc2-fedopt-50clients-mnist",
    hyperparameters=hyperparameters,  # pass args here
)

# Launch training
huggingface_estimator.fit(wait=False)


INFO:sagemaker.telemetry.telemetry_logging:SageMaker Python SDK will collect telemetry to help us better understand our user's needs, diagnose issues, and deliver additional features.
To opt out of telemetry, please disable via TelemetryOptOut parameter in SDK defaults config. For more information, refer to https://sagemaker.readthedocs.io/en/stable/overview.html#configuring-and-using-defaults-with-the-sagemaker-python-sdk.
INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: fingerprint-sc2-fedopt-50clients-mnist-2025-11-18-00-47-05-138


#### 3.2.2.2 FedAvg

In [29]:

from sagemaker.huggingface import HuggingFace

role = "arn:aws:iam::711387130895:role/service-role/SageMaker-ExecutionRole-20250525T120088"

# Define hyperparameters that match your argparse arguments
hyperparameters = {
    # --- Dataset and client setup ---
    "dataset_name": "mnist",
    "num_clients": 50,
    "frac_clients": 1.0,
    "dirichlet_alpha": 0.5,
    "test_size": 0.2,
    "poison_size": 0.35,

    # --- Training parameters ---
    "rounds": 12,
    "local_epochs": 3,
    "batch_size": 64, # from cifar10 64 
    "client_lr": 0.006,
    "client_momentum": 0.9,
    "weight_decay": 1e-4,

    # --- Server optimisation ---
    "server_opt": "fedavg",
    "server_lr": 0.35,

    # --- Fingerprinting ---
    "enable_fingerprinting": "1",          # or False if disabled
    "fingerprint_method": "sparse",         # choices: ["sparse", "dense"]
    "fingerprint_sparsity": 0.01,
    "target_dot_strength": 1.0,

    # --- Detection / Defence params ---
    "honest_fraction": 0.1,
    "detection_margin": 1.5,
    "seed": 42,
    "history_window": 5,
    "method": "fingerprint",                 # choices: ["label_flip", "backdoor","fingerprint"]
    "label_flip_alpha": 1.0,
    "backdoor_target_label": 1,
    "backdoor_patch_size": 15,
    "backdoor_intensity": 1.0,
    "tau_backdoor_threshold_statistical": tau_stat,
    "tau_backdoor_threshold_emprical": tau_emp,


    # --- Client selection ---
    # Lists must be passed as space-separated strings for SageMaker
    "targeted_clients": "0 1 2 3",
    "verified_clients": "2 3 4 5",
}


huggingface_estimator = HuggingFace(
    entry_point="main.py",   # script inside source_dir
    source_dir="s3://poison-detect-bucket/submission2/scripts/sourcedir.tar.gz",  # zipped dir
    role=role,
    instance_type="ml.g4dn.12xlarge",  # use quota for faster training
    instance_count=1,
    transformers_version="4.36.0",
    pytorch_version="2.1.0",
    py_version="py310",
    base_job_name="fingerprint-sc2-fedavg-50clients-mnist",
    hyperparameters=hyperparameters,  # pass args here
)

# Launch training
huggingface_estimator.fit(wait=False)


INFO:sagemaker.telemetry.telemetry_logging:SageMaker Python SDK will collect telemetry to help us better understand our user's needs, diagnose issues, and deliver additional features.
To opt out of telemetry, please disable via TelemetryOptOut parameter in SDK defaults config. For more information, refer to https://sagemaker.readthedocs.io/en/stable/overview.html#configuring-and-using-defaults-with-the-sagemaker-python-sdk.
INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: fingerprint-sc2-fedavg-50clients-mnist-2025-11-18-00-47-08-093
